## Market Basket Analysis With FP-Growth


In [3]:
#import thư viện
import pandas as pd
import numpy as np

In [4]:
#Đọc dữ liêu từ file
df = pd.read_csv('./data/input.csv', encoding='latin-1', dtype={'BillNo': str})

In [5]:
# Tiền xử lý sữ liệu
df.shape

(522064, 7)

In [6]:
df.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  float64
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [8]:
df.isnull().sum()

BillNo             0
Itemname        1455
Quantity           0
Date               0
Price              0
CustomerID    134041
Country            0
dtype: int64

In [9]:
itemCount = df.Itemname.nunique()
items = df.Itemname.unique()

print('Item count: ' + str(itemCount))
print(items)

Item count: 4185
['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER' ... 'lost'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE']


In [10]:
new_df = df.dropna()
new_df.shape

(388023, 7)

In [11]:
new_df.loc[df['Price'] < 0].shape

(0, 7)

In [12]:
most_sold_items = new_df['Itemname'].value_counts().head(20)
print('Most sold item: \n')
print(most_sold_items)

Most sold item: 

Itemname
WHITE HANGING HEART T-LIGHT HOLDER    1976
REGENCY CAKESTAND 3 TIER              1643
JUMBO BAG RED RETROSPOT               1591
ASSORTED COLOUR BIRD ORNAMENT         1391
PARTY BUNTING                         1369
LUNCH BAG RED RETROSPOT               1293
SET OF 3 CAKE TINS PANTRY DESIGN      1125
LUNCH BAG  BLACK SKULL.               1092
POSTAGE                               1043
PACK OF 72 RETROSPOT CAKE CASES       1031
PAPER CHAIN KIT 50'S CHRISTMAS        1002
LUNCH BAG SPACEBOY DESIGN              998
SPOTTY BUNTING                         994
LUNCH BAG CARS BLUE                    971
HEART OF WICKER SMALL                  967
NATURAL SLATE HEART CHALKBOARD         964
REX CASH+CARRY JUMBO SHOPPER           951
LUNCH BAG PINK POLKADOT                940
LUNCH BAG SUKI DESIGN                  924
JUMBO BAG PINK POLKADOT                883
Name: count, dtype: int64


In [13]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 388023 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      388023 non-null  object 
 1   Itemname    388023 non-null  object 
 2   Quantity    388023 non-null  int64  
 3   Date        388023 non-null  object 
 4   Price       388023 non-null  float64
 5   CustomerID  388023 non-null  float64
 6   Country     388023 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 23.7+ MB


In [15]:
transaction_list = []

for i in new_df['BillNo'].unique():
    tlist = list(set(new_df.loc[new_df['BillNo'] == i]['Itemname']))
    if len(tlist) > 0:
        transaction_list.append(tlist)

print(len(transaction_list))

18163


In [16]:
print(transaction_list[:10])

[['WHITE HANGING HEART T-LIGHT HOLDER', 'RED WOOLLY HOTTIE WHITE HEART.', 'SET 7 BABUSHKA NESTING BOXES', 'GLASS STAR FROSTED T-LIGHT HOLDER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER'], ['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT'], ['LOVE BUILDING BLOCK WORD', 'BOX OF VINTAGE JIGSAW BLOCKS', 'BOX OF VINTAGE ALPHABET BLOCKS', "POPPY'S PLAYHOUSE BEDROOM", 'ASSORTED COLOUR BIRD ORNAMENT', 'HOME BUILDING BLOCK WORD', 'FELTCRAFT PRINCESS CHARLOTTE DOLL', 'DOORMAT NEW ENGLAND', "POPPY'S PLAYHOUSE KITCHEN", 'BOX OF 6 ASSORTED COLOUR TEASPOONS', 'RECIPE BOX WITH METAL HEART', 'IVORY KNITTED MUG COSY'], ['YELLOW COAT RACK PARIS FASHION', 'BLUE COAT RACK PARIS FASHION', 'RED COAT RACK PARIS FASHION', 'JAM MAKING SET WITH JARS'], ['BATH BUILDING BLOCK WORD'], ['ROUND SNACK BOXES SET OF4 WOODLAND', 'CHARLOTTE BAG DOLLY GIRL DESIGN', 'LUNCH BOX I LOVE LONDON', 'PANDA AND BUNNIES STICKER SHEET', 'RED TOADSTOOL LED NIGHT LIGHT', 'VINTAG

In [17]:
# import thư viện khai phá luật kết hợp
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules,fpgrowth

In [18]:
te = TransactionEncoder()
te_ary = te.fit(transaction_list).transform(transaction_list)
df2 = pd.DataFrame(te_ary, columns=te.columns_)

In [19]:
df2.head(10)

,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
MIN_SUP = 0.02
MIN_CONF = 0.5
frequent_itemsets = fpgrowth(df2, min_support=MIN_SUP, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=MIN_CONF)

In [21]:
rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
51,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.028960,0.035952,0.023785,0.821293,22.844013,0.022743,5.394565,0.984743
49,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.035952,0.040412,0.027859,0.774885,19.174712,0.026406,4.262660,0.983196
53,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.028960,0.040412,0.022408,0.773764,19.146976,0.021238,4.241541,0.976038
67,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.034741,0.041238,0.025381,0.730586,17.716476,0.023949,3.558700,0.977515
48,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.040412,0.035952,0.027859,0.689373,19.174712,0.026406,3.103557,0.987765
...,...,...,...,...,...,...,...,...,...,...
8,(JUMBO BAG RED RETROSPOT),(LUNCH BAG RED RETROSPOT),0.086605,0.069702,0.023069,0.266370,3.821547,0.017032,1.268075,0.808331
42,(JUMBO BAG RED RETROSPOT),(JUMBO BAG STRAWBERRY),0.086605,0.035512,0.022408,0.258741,7.286074,0.019333,1.301149,0.944555
15,(JUMBO BAG RED RETROSPOT),(JUMBO SHOPPER VINTAGE RED PAISLEY),0.086605,0.043055,0.021582,0.249205,5.788129,0.017854,1.274577,0.905668
47,(REGENCY CAKESTAND 3 TIER),(ROSES REGENCY TEACUP AND SAUCER),0.089578,0.040412,0.021307,0.237861,5.885928,0.017687,1.259073,0.911778
